In [1]:
import sys
import csv

def read_from_csv(filename):
    csv.field_size_limit(sys.maxsize)
    with open(filename, 'r', newline='') as f:
        reader = csv.reader(f)
        next(reader)

        change_info_list = []
        for row in reader:
            change_info_list.append({
                "id": row[0],
                "gerrit_change_id": row[1],
                "project": row[2],
                "subject": row[3],
                "created": row[4],
                "updated": row[5],
                "status": row[6],
                "insertions": row[7],
                "deletions": row[8],
                "total_comment_count": row[9],
                "unresolved_comment_count": row[10],
                "messages": row[11],
                "comments": row[12],
                "added_methods": row[13],
                "removed_methods": row[14],
                "edited_methods": row[15],
                "diff_hunk": row[16]
            })

        return change_info_list
    
# Test the function
change_info_list = read_from_csv('dataset.csv')
len(change_info_list)

10663

In [2]:
# count all entries in change_info_list with status 'MERGED'
count = 0
merged_entries = []

for change_info in change_info_list:
    if change_info['status'] == 'MERGED':
        count += 1
        merged_entries.append(change_info)

print(count)

9765


In [3]:
# count all entries in change_info_list with status not 'MERGED'
count = 0
non_merged_entries = []

for change_info in change_info_list:
    if change_info['status'] != 'MERGED':
        count += 1
        non_merged_entries.append(change_info)

print(count)

898


In [4]:
# # select 898 random entries from merged_entries and create a balanced dataset with the non_merged_entries
import random

# Select 898 random entries from merged_entries
balanced_merged_entries = random.sample(merged_entries, 898)

# Combine with non_merged_entries
balanced_dataset = balanced_merged_entries + non_merged_entries

change_info_list = balanced_dataset

print(len(balanced_dataset))  # Should print 1796

1796


In [5]:
print(change_info_list[4]['added_methods'])

In [6]:
print(change_info_list[4]['added_methods'])

In [7]:
print(change_info_list[6]['comments'])

[{'filename': 'tests/org.eclipse.ui.tests/Eclipse JFace Tests/org/eclipse/jface/tests/viewers/Bug180504TreeViewerTest.java', 'message': 'whitespace', 'updated': '2014-08-04 09:57:42.000000000', 'commit_id': '0540356fb64da5c6660ee3c622faa1a683265511', 'unresolved': False}, {'filename': 'tests/org.eclipse.ui.tests/Eclipse JFace Tests/org/eclipse/jface/tests/viewers/Bug242231Test.java', 'message': 'Whitespace, please remove', 'updated': '2014-08-04 06:21:57.000000000', 'commit_id': '96159e44388c15e286b5fe46366e7239d7bd2ef4', 'unresolved': False}, {'filename': 'tests/org.eclipse.ui.tests/Eclipse JFace Tests/org/eclipse/jface/tests/viewers/Bug242231Test.java', 'message': 'Please remove, you already using @Override', 'updated': '2014-08-04 06:21:57.000000000', 'commit_id': '96159e44388c15e286b5fe46366e7239d7bd2ef4', 'unresolved': False}, {'filename': 'tests/org.eclipse.ui.tests/Eclipse JFace Tests/org/eclipse/jface/tests/viewers/Bug242231Test.java', 'message': 'Whitespace, please remove', 'u

In [8]:
print(change_info_list[4]['edited_methods'])

In [9]:
# for each row, convert the change_info_list[i]['messages'] and ['comments'] from a string to a list of dictionaries
for i in range(len(change_info_list)):
    change_info_list[i]['messages'] = eval(change_info_list[i]['messages'])
    change_info_list[i]['comments'] = eval(change_info_list[i]['comments'])

In [11]:
# create a new dataframe, with the same length of the change_info_list
import pandas as pd

df = pd.DataFrame(index=range(len(change_info_list)), columns=['message', 'status'])
# df = pd.DataFrame(index=range(len(balanced_dataset)), columns=['message', 'status'])

# MAX_LENGTH = 512
MAX_LENGTH = 4096

# edit the following columns: messages, comments, added_methods, removed_methods, edited_methods, diff_hunk
messages = []
for i in range(len(change_info_list)):
    messages_array = change_info_list[i]['messages']
    messages_list = ''
    for j in range(len(messages_array)):
        messages_list += str(j+1) + '. ' + messages_array[j]['message'] + '\n'

    # cap the maximal length of the messages_list to MAX_LENGTH characters (get the last MAX_LENGTH characters if the length is greater than MAX_LENGTH and start from the next line that starts with a number)
    if len(messages_list) > MAX_LENGTH:
        messages_list = messages_list[-MAX_LENGTH:]
        for k in range(MAX_LENGTH):
            if messages_list[k].isdigit():
                messages_list = messages_list[k:]
                break

    messages.append(messages_list)

comments = []
for i in range(len(change_info_list)):
    comments_array = change_info_list[i]['comments']
    comments_list = ''
    for j in range(len(comments_array)):
        comments_list += str(j+1) + '. '  + comments_array[j]['message'] + '\n'

    # cap the maximal length of the comments_list to MAX_LENGTH characters (get the last MAX_LENGTH characters if the length is greater than MAX_LENGTH and start from the next line that starts with a number)
    if len(comments_list) > MAX_LENGTH:
        comments_list = comments_list[-MAX_LENGTH:]
        for k in range(MAX_LENGTH):
            if comments_list[k].isdigit():
                comments_list = comments_list[k:]
                break

    comments.append(comments_list)

# make sure to limit every list to MAX_LENGTH characters
added_methods = []
removed_methods = []
edited_methods = []
diff_hunk = []
for i in range(len(change_info_list)):
    added_methods_list = change_info_list[i]['added_methods']
    removed_methods_list = change_info_list[i]['removed_methods']
    edited_methods_list = change_info_list[i]['edited_methods']
    diff_hunk_list = change_info_list[i]['diff_hunk']

    if len(added_methods_list) > MAX_LENGTH:
        added_methods_list = added_methods_list[:MAX_LENGTH]
    if len(removed_methods_list) > MAX_LENGTH:
        removed_methods_list = removed_methods_list[:MAX_LENGTH]
    if len(edited_methods_list) > MAX_LENGTH:
        edited_methods_list = edited_methods_list[:MAX_LENGTH]
    if len(diff_hunk_list) > MAX_LENGTH:
        diff_hunk_list = diff_hunk_list[:MAX_LENGTH]

    added_methods.append(added_methods_list)
    removed_methods.append(removed_methods_list)
    edited_methods.append(edited_methods_list)
    diff_hunk.append(diff_hunk_list)

# copy the colums listed below from change_info_list to df:
# project, subject, created, updated, insertions, deletions, total_comment_count, unresolved_comment_count
for i in range(len(change_info_list)):
    # text = "project: " + change_info_list[i]['project'] + "\nsubject: " + change_info_list[i]['subject'] + "\ncreated: " + change_info_list[i]['created'] + "\nupdated: " + change_info_list[i]['updated'] + "\ninsertions: " , change_info_list[i]['insertions'] + "\ndeletions: " + change_info_list[i]['deletions'] + "\ntotal comments: " + change_info_list[i]['total_comment_count'] + "\nunresolved comments: " + change_info_list[i]['unresolved_comment_count'] + "\nmessages: " + messages[i] + "\ncomments: " + comments[i] + "\nadded methods: " + added_methods[i] + "\nremoved methods: " + removed_methods[i] + "\nedited methods: " + edited_methods[i] + "\ndiff hunk:" + diff_hunk[i]
    # text = "project: " + change_info_list[i]['project'] + "\nsubject: " + change_info_list[i]['subject']
    text = "title:\n" + change_info_list[i]['subject'] + "\n\ndiff hunk:" + diff_hunk[i]
    df.loc[i] = [str(text), change_info_list[i]['status']]

print(df.head())

                                             message  status
0  title:\n[456994] Switch to 'Warning' level to ...  MERGED
1  title:\n[436359] Update Region size management...  MERGED
2  title:\nBug 546413 - [ActivityDiagram] Drop st...  MERGED
3  title:\n[api] moved DefaultSessionEditorPage f...  MERGED
4  title:\nBug 529435 - [launch] Deprecation of o...  MERGED


In [12]:
print(added_methods[4])
len(added_methods[4])


0

In [13]:
df['idx'] = df.index


# remove columns that we want to ignore for now
if 'project' in df.columns:
    df = df.drop(columns=['project'])

In [14]:
# preprocess the data for fine tuning
from datasets import ClassLabel, Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

train, test = train_test_split(df, test_size=0.2, stratify=df['status']) # this splits the data and shuffles the entries
train, val = train_test_split(train, test_size=0.2, stratify=train['status'])

print(len(train))
# # balance the training set
# positive_samples = train[train['status'] == 'MERGED']
# negative_samples = train[train['status'] != 'MERGED']
# if len(positive_samples) > len(negative_samples):
#     positive_samples = resample(positive_samples, replace=False, n_samples=len(negative_samples), random_state=42)
# else:
#     negative_samples = resample(negative_samples, replace=False, n_samples=len(positive_samples), random_state=42)

# balanced_train = pd.concat([positive_samples, negative_samples], ignore_index=True)

# train_dataset = Dataset.from_pandas(balanced_train).shuffle(seed=42)

train_dataset = Dataset.from_pandas(train).shuffle(seed=42)
val_dataset = Dataset.from_pandas(val)
test_dataset = Dataset.from_pandas(test)

raw_datasets = DatasetDict({
    'train': train_dataset, 
    'validation': val_dataset, 
    'test': test_dataset
})

print(len(raw_datasets['train']))
# Normalize the 'status' column to lowercase
def recategorize_status(example):
    # Normalize the status to lowercase
    normalized_status = example['status'].lower()
    # Recategorize status
    if normalized_status == 'merged':
        example['status'] = 'merged'
    else:
        example['status'] = 'not_merged'
    return example

# Normalize and recategorize status in all splits
for split in raw_datasets.keys():
    raw_datasets[split] = raw_datasets[split].map(recategorize_status)
    
# create the ClassLabel feature
class_label_feature = ClassLabel(num_classes=2, names=['not_merged', 'merged'])

for split in raw_datasets.keys():
    raw_datasets[split] = raw_datasets[split].cast_column('status', class_label_feature)

# rename the 'status' column to 'label'
for split in raw_datasets.keys():
    raw_datasets[split] = raw_datasets[split].rename_column('status', 'label')

# remove the __index_level... columns if it exists (just if cell gets ran multiple times)
if '__index_level_0__' in raw_datasets['train'].column_names:
    raw_datasets['train'] = raw_datasets['train'].remove_columns('__index_level_0__')

if '__index_level_0__' in raw_datasets['test'].column_names:
    raw_datasets['test'] = raw_datasets['test'].remove_columns('__index_level_0__')
    
if '__index_level_0__' in raw_datasets['validation'].column_names:
    raw_datasets['validation'] = raw_datasets['validation'].remove_columns('__index_level_0__')

raw_datasets


1148
1148


Map:   0%|          | 0/1148 [00:00<?, ? examples/s]

Map:   0%|          | 0/288 [00:00<?, ? examples/s]

Map:   0%|          | 0/360 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/1148 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/288 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/360 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['message', 'label', 'idx'],
        num_rows: 1148
    })
    validation: Dataset({
        features: ['message', 'label', 'idx'],
        num_rows: 288
    })
    test: Dataset({
        features: ['message', 'label', 'idx'],
        num_rows: 360
    })
})

In [15]:
# print random entry from training set
print(raw_datasets['train'][5])
print(len(raw_datasets['train'][5]['message']))

{'message': 'title:\n417451: Allow config the number of executors and max http connections\n\ndiff hunk:From 6f590f60aa854af631bff098ed89fd50bc6ebdd4 Mon Sep 17 00:00:00 2001\nFrom: Tomasz Zarna <tomasz.zarna@tasktop.com>\nDate: Fri, 07 Mar 2014 19:16:28 +0100\nSubject: [PATCH] 417451: Allow config the number of executors and max http connections\n\nChange-Id: Id8437323a6bc0f912a5b164e3f94837152f964d7\nTask-Url: https://bugs.eclipse.org/bugs/show_bug.cgi?id=417451\nSigned-off-by: Tomasz Zarna <tomasz.zarna@tasktop.com>\n---\n\ndiff --git a/org.eclipse.mylyn.commons.core/src/org/eclipse/mylyn/commons/core/net/NetUtil.java b/org.eclipse.mylyn.commons.core/src/org/eclipse/mylyn/commons/core/net/NetUtil.java\nindex 7272a42..5bc0c90 100644\n--- a/org.eclipse.mylyn.commons.core/src/org/eclipse/mylyn/commons/core/net/NetUtil.java\n+++ b/org.eclipse.mylyn.commons.core/src/org/eclipse/mylyn/commons/core/net/NetUtil.java\n@@ -44,10 +44,18 @@\n  */\n public class NetUtil {\n \n+\tprivate static f

In [16]:
# show a slice of the dataset
raw_datasets['train'][:30]

{'message': ['title:\nBug 472224 - Update Factory.getPackedAnnotationBindings to never return null entries.\n\ndiff hunk:From 31d5f22b58d50463acba53b1e4e7ddb99bb3d73d Mon Sep 17 00:00:00 2001\nFrom: Stefan Xenos <sxenos@gmail.com>\nDate: Mon, 13 Jul 2015 12:44:53 -0700\nSubject: [PATCH] Bug 472224 - Update Factory.getPackedAnnotationBindings to never return null entries.\n\nChange-Id: I0ad3e3128c32d05d99c14000c63fd1a8ad354e0c\nSigned-off-by: Stefan Xenos <sxenos@gmail.com>\n---\n\ndiff --git a/org.eclipse.jdt.compiler.apt/src/org/eclipse/jdt/internal/compiler/apt/model/Factory.java b/org.eclipse.jdt.compiler.apt/src/org/eclipse/jdt/internal/compiler/apt/model/Factory.java\nindex d18616b..b6c5019 100644\n--- a/org.eclipse.jdt.compiler.apt/src/org/eclipse/jdt/internal/compiler/apt/model/Factory.java\n+++ b/org.eclipse.jdt.compiler.apt/src/org/eclipse/jdt/internal/compiler/apt/model/Factory.java\n@@ -769,7 +769,8 @@\n \t}\n \n \t/* Wrap repeating annotations into their container, return a

In [17]:
raw_datasets['test'][:30]

{'message': ['title:\n434163: Dynamic Domain Model References https://bugs.eclipse.org/bugs/show_bug.cgi?id=434163\n\ndiff hunk:From 9765eb321abcb4af242d98a9af11e30b8aaafb86 Mon Sep 17 00:00:00 2001\nFrom: eneufeld <neufeld.eugen@googlemail.com>\nDate: Tue, 20 May 2014 10:20:51 +0200\nSubject: [PATCH] 434163: Dynamic Domain Model References https://bugs.eclipse.org/bugs/show_bug.cgi?id=434163\n\nChange-Id: I639d37f0a617e5ad10c2ad77046b0d24565d6f73\nSigned-off-by: eneufeld <neufeld.eugen@googlemail.com>\n---\n\ndiff --git a/bundles/org.eclipse.emf.ecp.ide.editor.view/src/org/eclipse/emf/ecp/ide/editor/view/ViewEditorPart.java b/bundles/org.eclipse.emf.ecp.ide.editor.view/src/org/eclipse/emf/ecp/ide/editor/view/ViewEditorPart.java\nindex 05d1c59..7cfdd81 100644\n--- a/bundles/org.eclipse.emf.ecp.ide.editor.view/src/org/eclipse/emf/ecp/ide/editor/view/ViewEditorPart.java\n+++ b/bundles/org.eclipse.emf.ecp.ide.editor.view/src/org/eclipse/emf/ecp/ide/editor/view/ViewEditorPart.java\n@@ -12,

In [18]:
raw_datasets['train'].features

{'message': Value(dtype='string', id=None),
 'label': ClassLabel(names=['not_merged', 'merged'], id=None),
 'idx': Value(dtype='int64', id=None)}

In [18]:
# import tensorflow as tf
# tf.keras.backend.clear_session()  # This clears the session


In [19]:
# count how many entries in the training set are 'merged' and 'not_merged'
count = 0
for entry in raw_datasets['train']:
    if entry['label'] == 1:
        count += 1

print(count)
print(len(raw_datasets['train']) - count)

574
574


In [20]:
# count how many entries in the test set are 'merged' and 'not_merged'
count = 0
for entry in raw_datasets['validation']:
    if entry['label'] == 1:
        count += 1

print(count)
print(len(raw_datasets['validation']) - count)

144
144


In [21]:
# tokenize the data
from transformers import AutoTokenizer

# Load the BERT tokenizer
checkpoint = 'bert-base-cased'
# checkpoint = 'google-bert/bert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example['message'], padding="max_length", truncation=True)

# tokenize the training and validation sets
# tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
print(tokenized_datasets.column_names)

/Users/eric/uzh/pr-prediction/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Map:   0%|          | 0/1148 [00:00<?, ? examples/s]

Map:   0%|          | 0/288 [00:00<?, ? examples/s]

Map:   0%|          | 0/360 [00:00<?, ? examples/s]

{'train': ['message', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'], 'validation': ['message', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'], 'test': ['message', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask']}


In [22]:
# from transformers import DataCollatorWithPadding

# data_collator = DataCollatorWithPadding(tokenizer)

In [22]:
import torch

torch.cuda.is_available()

False

In [23]:
print(torch.__version__)

2.3.0


In [24]:
import accelerate
print(accelerate.__version__)

0.30.1


In [25]:
from transformers import AutoModelForSequenceClassification, TrainingArguments

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

training_args = TrainingArguments("test-trainer")

/Users/eric/uzh/pr-prediction/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
import numpy as np
import evaluate
from sklearn.metrics import roc_auc_score

accuracy_metric = evaluate.load('accuracy')
f1_metric = evaluate.load('f1')
precision_metric = evaluate.load('precision')
recall_metric = evaluate.load('recall')
# metric = evaluate.load('f1')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Calculate each metric
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average='macro')
    precision = precision_metric.compute(predictions=predictions, references=labels, average='macro', zero_division=0)
    recall = recall_metric.compute(predictions=predictions, references=labels, average='macro')
 
    # For AUC-ROC, we need to calculate it using sklearn if it's a binary classification
    if logits.shape[1] == 2:  # binary classification
        # Compute probabilities as softmax of logits
        probabilities = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)
        # Select probability of positive class
        prob_positive = probabilities[:, 1]
        auc_roc = roc_auc_score(labels, prob_positive)
    else:
        auc_roc = None  # AUC-ROC is not applicable for non-binary classification directly

    result = {
        "accuracy": accuracy['accuracy'],
        "f1": f1['f1'],
        "precision": precision['precision'],
        "recall": recall['recall'],
    }

  # Add AUC-ROC if it's computed
    if auc_roc is not None:
        result["auc_roc"] = auc_roc

    return result

    # return metric.compute(predictions=predictions, references=labels)

from torch import zero_
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [27]:
# small_train_dataset = tokenized_datasets["train"].shuffle(seed=33).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(300))
small_train_dataset = tokenized_datasets["train"]
# small_eval_dataset = tokenized_datasets["test"]

print(len(small_train_dataset))
print(len(small_eval_dataset))

1148
300


In [28]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    # data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [29]:
trainer.train()

  0%|          | 0/432 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

{'eval_loss': 0.6443676352500916, 'eval_accuracy': 0.62, 'eval_f1': 0.5981198589894242, 'eval_precision': 0.6415343915343915, 'eval_recall': 0.614214552570717, 'eval_auc_roc': 0.6845756982743284, 'eval_runtime': 9.1206, 'eval_samples_per_second': 32.893, 'eval_steps_per_second': 4.166, 'epoch': 1.0}


  0%|          | 0/38 [00:00<?, ?it/s]

{'eval_loss': 0.583109438419342, 'eval_accuracy': 0.5933333333333334, 'eval_f1': 0.5274736624664326, 'eval_precision': 0.7578505086245024, 'eval_recall': 0.6037181996086105, 'eval_auc_roc': 0.7250044476071873, 'eval_runtime': 8.9629, 'eval_samples_per_second': 33.471, 'eval_steps_per_second': 4.24, 'epoch': 2.0}


  0%|          | 0/38 [00:00<?, ?it/s]

{'eval_loss': 0.5542563199996948, 'eval_accuracy': 0.6133333333333333, 'eval_f1': 0.6022857142857143, 'eval_precision': 0.6357230392156863, 'eval_recall': 0.6182173990393168, 'eval_auc_roc': 0.7312755737413272, 'eval_runtime': 8.9709, 'eval_samples_per_second': 33.441, 'eval_steps_per_second': 4.236, 'epoch': 3.0}
{'train_runtime': 416.5764, 'train_samples_per_second': 8.267, 'train_steps_per_second': 1.037, 'train_loss': 0.6267759535047743, 'epoch': 3.0}


TrainOutput(global_step=432, training_loss=0.6267759535047743, metrics={'train_runtime': 416.5764, 'train_samples_per_second': 8.267, 'train_steps_per_second': 1.037, 'total_flos': 906154474659840.0, 'train_loss': 0.6267759535047743, 'epoch': 3.0})

In [30]:
evaluation_results = trainer.evaluate(eval_dataset=tokenized_datasets['validation'])
print(evaluation_results)

  0%|          | 0/36 [00:00<?, ?it/s]

{'eval_loss': 0.564671516418457, 'eval_accuracy': 0.6354166666666666, 'eval_f1': 0.6216168870981869, 'eval_precision': 0.6585455366721247, 'eval_recall': 0.6354166666666666, 'eval_auc_roc': 0.7451774691358024, 'eval_runtime': 8.6554, 'eval_samples_per_second': 33.274, 'eval_steps_per_second': 4.159, 'epoch': 3.0}
